In [1]:
%load_ext autoreload
%autoreload 2

In [71]:
import cv2
from collections import OrderedDict
import torch
import numpy as np
import albumentations as A
from albumentations.pytorch import ToTensorV2
from matplotlib import pyplot as plt
from anomalib.pre_processing.pre_process import get_transforms, PreProcessor
import torchvision.models.detection as detection
from anomalib.data import InferenceDataset
from torch.utils.data import DataLoader
from torchvision.transforms import ToPILImage
import torch.nn.functional as F

from anomalib.models.rbad.region_extractor import RegionExtractor as RegionExtractor2
from anomalib.models.rbad.region import RegionExtractor as RegionExtractor1
from anomalib.models.rbad.feature import FeatureExtractor as FeatureExtractor1
from anomalib.models.rbad.feature_extractor import FeatureExtractor as FeatureExtractor2

In [73]:
filename = "150.tif"
image = cv2.imread(filename)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Transformations.
transforms = get_transforms(config=A.Compose([A.Normalize(mean=0.0, std=1.0), ToTensorV2()]))
pre_process = PreProcessor(config=transforms)

# Get the data via dataloader
dataset = InferenceDataset(path=filename, pre_process=pre_process)
dataloader = DataLoader(dataset)
i, data = next(enumerate(dataloader))

# Create the region extractor.
stage="rpn"
use_original = True
region_extractor1 = RegionExtractor1(stage=stage, use_original=use_original).eval().cuda()
region_extractor2 = RegionExtractor2(stage=stage, use_original=use_original).eval().cuda()

# Forward-Pass the input
boxes1 = region_extractor1([image])
boxes2 = region_extractor2(data["image"].cuda())

# Feature Extractor
feature_extractor1 = FeatureExtractor1().eval().cuda()
feature_extractor2 = FeatureExtractor2().eval().cuda()
features1 = feature_extractor1(image, boxes1[0])
features2 = feature_extractor2(data["image"].cuda(), boxes2)


In [94]:
image = cv2.imread(filename)
boxes = boxes1[0]
image1, scale1 = feature_extractor1.transform(image)
boxes_pt = torch.tensor(boxes)
rois1 = torch.cat((torch.zeros(boxes_pt.size(0), 1), boxes_pt), 1).unsqueeze(0).to(feature_extractor1.device)
rois1 *= scale1
base_feats1 = feature_extractor1.head_module(image1)
rcnn_feats1 = feature_extractor1.rcnn_module(base_feats1, rois1)

In [ ]:
image2, scale2 = feature_extractor2.transform(data["image"].cuda())

# Process RoIs.
# Convert list of boxes to batch format.
rois = [box.unsqueeze(0) for box in boxes]
rois = torch.cat(boxes, dim=0)
# Add zero column for the scores.
rois = F.pad(input=rois, pad=(1, 0, 0, 0), mode="constant", value=0)
# Scale the RoIs based on the the new image size.
rois *= scale

# Extract RCNN features.
backbone_features = self.backbone(images)
rcnn_features = self.rcnn_module(backbone_features, rois)

# Feature Extractor

In [63]:
# image = cv2.imread(filename)
# # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
# print(image.shape)

# in_shape = image.shape
# in_size_min = np.min(in_shape[0:2])
# in_size_max = np.max(in_shape[0:2])

# target_size = 600
# max_size = 1000

# scale = float(target_size) / float(in_size_min)
# if np.round(scale * in_size_max) > max_size:
#     print("WARNING: cfg.MAX_SIZE exceeded. Using a different scaling ratio")
#     scale = float(max_size) / float(in_size_max)

# # TODO: Use the scale version.
# image = cv2.resize(image, None, None, fx=scale, fy=scale, interpolation=cv2.INTER_LINEAR)
# image = cv2.resize(image, (904, 600), interpolation=cv2.INTER_LINEAR)
# image = image.astype(np.float32)
# image /= 255.0
# image = torch.from_numpy(image)
# image = image.unsqueeze(0)
# image = image.permute(0, 3, 1, 2)
# image = feature_extractor1.normalizer.normalize(image)

# if torch.cuda.is_available():
#     image = image.cuda()

In [52]:
image.shape

torch.Size([1, 3, 600, 904])

In [47]:
input_tensor = data["image"].cuda()
input_tensor.shape

torch.Size([1, 3, 158, 238])

In [36]:
boxes = F.pad(input=boxes_pt, pad=(1, 0, 0, 0), mode="constant", value=0).unsqueeze(dim=0)
boxes.shape

torch.Size([1, 24, 5])

In [11]:
torch.cat((torch.zeros(boxes_pt.size(0), 1), boxes_pt), 1).shape

torch.Size([24, 5])

torch.Size([1, 24, 5])

In [9]:
boxes_pt = torch.tensor(boxes1[0])
rois = torch.cat((torch.zeros(boxes_pt.size(0), 1), boxes_pt), 1).unsqueeze(0).to(input_tensor.device)
boxes_pt.shape, rois.shape


(torch.Size([24, 4]), torch.Size([1, 24, 5]))

In [39]:
# # Apply the feature extractor transforms 
# height, width = input_tensor.shape[2:]
# shorter_image_size, longer_image_size = min(height, width), max(height, width)
# target_image_size, max_image_size = 600, 1000

# scale = target_image_size / shorter_image_size
# if round(scale * longer_image_size) > max_image_size:
#     print("WARNING: cfg.MAX_SIZE exceeded. Using a different scaling ratio")
#     scale = max_image_size / longer_image_size

# # TODO: Use scale_factor
# # resized_image = F.interpolate(input_tensor, scale_factor=scale, mode="bilinear", align_corners=False)
# resized_tensor = F.interpolate(input_tensor, size=(600, 904), mode="bilinear", align_corners=False)

# # Apply the same transformation as the original model.
# mean = torch.tensor([0.485, 0.456, 0.406]).unsqueeze(0).unsqueeze(2).unsqueeze(3).to(input_tensor.device)
# std = torch.tensor([0.229, 0.224, 0.225]).unsqueeze(0).unsqueeze(2).unsqueeze(3).to(input_tensor.device)
# normalized_tensor = (resized_tensor - mean) / std

# # NOTE: Return the normalized_tensor and scale here.

In [56]:
feature_extractor1.transform(image)

(tensor([[[[-0.5938, -0.5938, -0.6109,  ..., -1.0219, -0.9877, -0.9877],
           [-0.5938, -0.5938, -0.6109,  ..., -1.0219, -0.9877, -0.9877],
           [-0.6623, -0.6623, -0.6623,  ..., -0.9877, -0.9534, -0.9534],
           ...,
           [-0.1314, -0.1314, -0.1314,  ...,  1.0331,  1.0673,  1.0673],
           [-0.1314, -0.1314, -0.1314,  ...,  1.0502,  1.0844,  1.0844],
           [-0.1314, -0.1314, -0.1314,  ...,  1.0502,  1.0844,  1.0844]],
 
          [[-0.4776, -0.4776, -0.4951,  ..., -0.9153, -0.8803, -0.8803],
           [-0.4776, -0.4776, -0.4951,  ..., -0.9153, -0.8803, -0.8803],
           [-0.5476, -0.5476, -0.5476,  ..., -0.8803, -0.8452, -0.8452],
           ...,
           [-0.0049, -0.0049, -0.0049,  ...,  1.1856,  1.2206,  1.2206],
           [-0.0049, -0.0049, -0.0049,  ...,  1.2031,  1.2381,  1.2381],
           [-0.0049, -0.0049, -0.0049,  ...,  1.2031,  1.2381,  1.2381]],
 
          [[-0.2532, -0.2532, -0.2707,  ..., -0.6890, -0.6541, -0.6541],
           [-

In [50]:
feature_extractor2.transform(input_tensor)

(tensor([[[[-0.5938, -0.5938, -0.6019,  ..., -1.0310, -0.9877, -0.9877],
           [-0.5938, -0.5938, -0.6019,  ..., -1.0310, -0.9877, -0.9877],
           [-0.6616, -0.6616, -0.6689,  ..., -0.9999, -0.9497, -0.9497],
           ...,
           [-0.1341, -0.1341, -0.1327,  ...,  1.0198,  1.0763,  1.0763],
           [-0.1314, -0.1314, -0.1287,  ...,  1.0275,  1.0844,  1.0844],
           [-0.1314, -0.1314, -0.1287,  ...,  1.0275,  1.0844,  1.0844]],
 
          [[-0.4776, -0.4776, -0.4859,  ..., -0.9246, -0.8803, -0.8803],
           [-0.4776, -0.4776, -0.4859,  ..., -0.9246, -0.8803, -0.8803],
           [-0.5469, -0.5469, -0.5543,  ..., -0.8928, -0.8414, -0.8414],
           ...,
           [-0.0077, -0.0077, -0.0062,  ...,  1.1721,  1.2298,  1.2298],
           [-0.0049, -0.0049, -0.0021,  ...,  1.1799,  1.2381,  1.2381],
           [-0.0049, -0.0049, -0.0021,  ...,  1.1799,  1.2381,  1.2381]],
 
          [[-0.2532, -0.2532, -0.2615,  ..., -0.6982, -0.6541, -0.6541],
           [-

In [64]:
# Old Forward-Pass
image = cv2.imread(filename)
image, scale = feature_extractor1.transform(image)
boxes_pt = torch.tensor(boxes1[0])
rois = torch.cat((torch.zeros(boxes_pt.size(0), 1), boxes_pt), 1).unsqueeze(0).to(feature_extractor1.device)
rois *= scale

base_feats = feature_extractor1.head_module(image)
rcnn_feats = feature_extractor1.rcnn_module(base_feats, rois)

rcnn_feats.shape

torch.Size([24, 4096])

In [68]:
# Forward-Pass
# TODO: Replace this with the new transforms.
image = cv2.imread(filename)
image, scale = feature_extractor1.transform(image)

# TODO: Remove the following line
boxes = [box.unsqueeze(0) for box in boxes2]
boxes = torch.cat(boxes, dim=0)
boxes = F.pad(input=boxes, pad=(1, 0, 0, 0), mode="constant", value=0)
boxes *= scale

base_feats = feature_extractor1.head_module(image)
rcnn_feats = feature_extractor1.rcnn_module(base_feats, boxes)


In [69]:
rois.shape, boxes.shape

(torch.Size([1, 24, 5]), torch.Size([1, 24, 5]))